In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sns

In [2]:
data = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_190330.txt")

In [8]:
data.head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/23/2019,00:00:00,REGULAR,6989774,2370411
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/23/2019,04:00:00,REGULAR,6989795,2370413
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/23/2019,08:00:00,REGULAR,6989813,2370436
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/23/2019,12:00:00,REGULAR,6989924,2370512
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/23/2019,16:00:00,REGULAR,6990200,2370573
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/23/2019,20:00:00,REGULAR,6990562,2370623
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/24/2019,00:00:00,REGULAR,6990734,2370648
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/24/2019,04:00:00,REGULAR,6990758,2370653
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/24/2019,08:00:00,REGULAR,6990772,2370676
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/24/2019,12:00:00,REGULAR,6990860,2370731


In [10]:
data[data.STATION=='59 ST'].LINENAME.unique()

array(['NQR456W', 'NRW', '456NQRW'], dtype=object)

In [7]:
data[data.STATION=='59 ST'].DIVISION.unique()

array(['BMT', 'IRT'], dtype=object)

In [11]:
data[data.STATION=='59 ST'].UNIT.unique()

array(['R051', 'R212', 'R050'], dtype=object)

In [15]:
data[(data['C/A']=='A002') & (data['UNIT']=='R051')].SCP.unique()

array(['02-00-00', '02-00-01', '02-03-00', '02-03-01', '02-03-02',
       '02-03-03', '02-03-04', '02-03-05', '02-03-06', '02-05-00',
       '02-05-01', '02-06-00'], dtype=object)

In [16]:
data[(data['C/A']=='A002') & (data['UNIT']=='R051')].DIVISION.unique()

array(['BMT'], dtype=object)